### Getting started with the Ollama library ###

In [1]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from textwrap import dedent

# Ollama
import ollama
from pydantic import BaseModel

# Financial news data for this notebook
from gnews import GNews

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
# print(f'Package version: {llmt.__version__}')

In [2]:
# Data directory and files
data_dir = os.path.join(os.environ.get('HOME'), 'data')
print(f'Data directory: {data_dir}')

Data directory: /app/data


In [3]:
# Create a connection to Ollama
client = ollama.Client(host='http://ollama:11434')
try:
    client = ollama.Client(host='http://ollama:11434')
    models = client.list().models
    if models:
        print("Available models:")
        for model in models:
            print(f"- {model['model']}")
    else:
        print("No models found.")
except Exception as e:
    print(f"Error: {e}")

04/07/2025 05:38:23 PM - HTTP Request: GET http://ollama:11434/api/tags "HTTP/1.1 200 OK"


Available models:
- llama3.3:latest


### Ollama getting started ###

In [4]:
# Fetch new articles
google_news = GNews()
news = google_news.get_news('NVDA')

In [5]:
# Extract a few news titles
news_titles = [article.get('title') for article in news[:6]]
print(*news_titles, sep='\n\n')

Nvidia Stock (NVDA) Named a Top Semiconductor Pick by Bank of America - TipRanks

Will Nvidia Stock (NVDA) Recover from Its Tariff Fall? - TipRanks

Ives: Investors With Long-Term Horizons Should Buy Nvidia (NVDA) Stock - Yahoo Finance

Live Updates: Magnificent 7 Stocks Apple (Nasdaq: AAPL), NVIDIA (Nasdaq: NVDA), and Microsoft (Nasdaq: MSFT) See Share Prices Sink Again Monday - 24/7 Wall St.

Buying Opportunities, Including NVDA, Are Starting to Arise, Veteran Investor Says - Yahoo Finance

Zacks Investment Ideas feature highlights: Palantir and NVIDIA - TradingView


In [6]:
class NewsAnalysis(BaseModel):
    sentiment: str
    future_looking: bool

system_message = f"""You are a powerful AI that can analyze the sentiment of news messages and give advice on specific stocks."""
model = 'llama3.3'

for title in news_titles:
    messages = [{'role': 'user',
                 'content': 
                 f"""Analyze the following title for sentiment (positive, negative, or neutral)
                 and whether it provides future-looking financial insight, predictions, or guidance on 
                 whether to buy/hold/sell the stock (True or False): {title}
                 """,}]
    response = client.chat(messages=messages, model=model, format=NewsAnalysis.model_json_schema())
    # Parse the response into the NewsAnalysis model
    sentiment_analysis = NewsAnalysis.model_validate_json(response['message']['content'])

04/07/2025 05:39:08 PM - HTTP Request: POST http://ollama:11434/api/chat "HTTP/1.1 500 Internal Server Error"


ResponseError: model requires more system memory (45.3 GiB) than is available (13.4 GiB) (status code: 500)